# 第二次小作业内容  

**1. 作业内容：**  

* 使用多个自变量(两个或两个以上，自行选择)对恋爱情况(因变量)建立logistic回归模型  

**2. 具体要求：**  
    重复课上的workflow，包括:  
		
    1. 模型定义(要求附上基本的文字解释，格式参考附后）  
    
    2. MCMC采样，模型诊断图，后验参数解释  
        * 使用az.plot_trace绘制模型诊断图  
        * 对后验参数结果在模型中的意义进行文字解释  
        * 绘制后验回归模型(使用az.plot_hdi)  

    3. 使用定义好的模型，对新站点的结果进行预测并评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    4. 对本数据集的预测结果进行评估  
        * 按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释  
    
    5. 与课上的模型(log_model1:自变量为回避倾向; log_model2:自变量为性别)进行模型比较  

**3. 作业截止时间：12.19**  

**4. 作业提交于和鲸平台**  


* 注1：所有的文字解释仅需增加在notebook中，无需提交额外的文档  

* 注2：上述要求请使用pymc语法实现  

* 注3：在数据预处理时注意缺失值  

* 注4：每个小组内成员所使用的站点数据是相同的，但该作业为个人独立提交，小组内可以互相讨论  

| 组别  | 建立模型使用的站点 | 对新数据进行预测时使用的站点  |  
|---------|------------------------------|-----------------------|  
| 1 | METU   | UCSB  |  
| 2 | Oxford | Poland |  
| 3 | Serbia | VCU |  
| 4 | VCU |Oxford |  
| 5 | UCSB |Serbia |  


**模型定义文字描述格式参考：**  

1. 自变量：xx  

2. 因变量：xx  

3. 数据关系：  

$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1 & {\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}}}{1 + e^{\beta_0 + \beta_1 X_{i1}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N()  \\  
               & & \beta_1  & \sim N(). \\  
\end{array}  
$$  

**研究问题**  

我关注的研究问题是：**家庭依恋**和**外界平均气温**如何影响个体的**恋爱情况**？  

* 研究假设：生活在平均气温低且家庭依恋低的个体可能更倾向于与伴侣建立深入的情感联系，更容易建立恋爱关系。  

> * 数据来源: Hu, C.-P. et al. (2018). Raw data from the Human Penguin Project. Open Science Framework. https://doi.org/10.17605/OSF.IO/H52D3  

# 导入函数、加载数据

In [1]:
# 导入 pymc 模型包，和 arviz 等分析工具 
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 通过 pd.read_csv 加载数据 Data_Sum_HPP_Multi_Site_Share.csv
df_raw = pd.read_csv('/home/mw/input/bayes20238001/Data_Sum_HPP_Multi_Site_Share.csv')


# 选取UCSB站点的数据
df = df_raw[df_raw["Site"] == "UCSB"]

# 选取本节课涉及的变量
df = df[["romantic","avgtemp", "attachhome","avoidance_r","sex"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df["romantic"] =  np.where(df['romantic'] == 2, 0, 1)
#0 表示男性，1表示女性
df["sex"] =  np.where(df['sex'] == 1, 0, 1)

#设置索引
df["index"] = range(len(df))
df = df.set_index("index")

In [3]:
# 检查数据中是否含有任何缺失值：
df.isnull().values.any()

False

In [4]:
# 展示数据
df.head(10)

,romantic,avgtemp,attachhome,avoidance_r,sex
index,,,,,
0,0,36.111111,4.666667,-1.083724,1
1,1,36.222222,4.444444,-1.698926,0
2,0,36.500000,4.444444,0.813149,1
3,0,36.527778,3.666667,-0.212188,1
4,1,36.638889,4.555556,-1.134991,0
5,0,36.694444,4.555556,1.172017,0
6,0,36.833333,4.000000,-1.186257,0
7,0,36.944444,4.444444,0.505548,0
8,1,36.972222,4.888889,0.351747,1


In [5]:
# 绘制散点图
sns.scatterplot(data=df,
                x="attachhome",
                y="avgtemp",
                hue="romantic",
                alpha=0.6)
# 设置x轴标题
plt.xlabel("attachhome")
sns.despine()

<Figure size 640x480 with 1 Axes>

# 模型定义  

1. 自变量： $\text{外界平均气温} X_{i1}、\text{家庭依恋} X_{i2}$  

2. 因变量：$\text{恋爱情况} Y_{i}$  

3. 数据关系：  

$$  
\begin{array}{lcrl}  
\text{data:} & \hspace{.01in} & Y_i|\beta_0,\beta_1,\beta_2 & {\sim} \text{Bern}(\pi_i) \;\; \text{ with } \;\; \pi_i = \frac{e^{\beta_0 + \beta_1 X_{i1}+\beta_2 X_{i2}}}{1 + e^{\beta_0 + \beta_1 X_{i1}+\beta_2 X_{i2}}} \\  
\text{priors:} & & \beta_{0}  &  \sim N(0.0.5^2)  \\  
               & & \beta_1  & \sim N(0,0.5^2) \\  
							 & & \beta_2  & \sim N(0,0.5^2).\\  
\end{array}  
$$ 

In [6]:
with pm.Model() as log_model1:
    # 此处对coords的定义方式进行了更改，因为后续我们需要进行对新数据的预测
    # 因此将维度定义成可更改的
    log_model1.add_coord('obs_id',df.index, mutable=True)
    avgtemp = pm.MutableData("avgtemp", df.avgtemp, dims="obs_id")
    attachhome = pm.MutableData("attachhome", df.attachhome, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)         #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)          #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * avgtemp + beta_2 * attachhome, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [7]:
# 可视化模型（贝叶斯变量因果图）
# 我们使用 pymc 自带的 `model_to_graphviz` 方法来可视化模型中各变量的因果关系。
pm.model_to_graphviz(log_model1)

#  MCMC采样，模型诊断图&后验参数解释  

In [8]:
#===========================
#     注意！！！以下代码可能需要运行1-2分钟左右
#===========================
with log_model1:
    # MCMC 近似后验分布
    log_model1_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 74 seconds.


## 使用az.plot_trace绘制模型诊断图 

In [9]:
az.plot_trace(log_model1_trace,
              var_names=["beta_0","beta_1","beta_2"],
              figsize=(15,8),
              compact=False)
plt.show()

<Figure size 1500x800 with 6 Axes>

## 对后验参数结果在模型中的意义进行文字解释   

In [10]:
az.summary(log_model1_trace, var_names=["beta_0","beta_1","beta_2"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,-0.017,0.497,-0.962,0.893,0.005,0.004,9063.0,7670.0,1.0
beta_1,-0.013,0.031,-0.071,0.046,0.000,0.000,7086.0,8205.0,1.0
beta_2,0.098,0.255,-0.389,0.565,0.003,0.002,7127.0,8011.0,1.0


In [11]:
# 通过 np.exp 将 beta 参数进行转换
az.plot_posterior(log_model1_trace, var_names=["beta_0","beta_1","beta_2"], transform = np.exp)
plt.show()

<Figure size 2208x552 with 3 Axes>

以下的结果显示：  
- $\beta_0 = -0.02$，那么 $e^{\beta_0} = 1.1$， 表明 X1 和 X2 为 0时，个体恋爱的可能性为 1.1。  
- $\beta_1 = -0.01$， $e^{\beta_1} = 0.86$， 表明外界平均气温每增加 1个单位，个体恋爱的发生比变为之前的 0.99倍。  
- $\beta_2 = 0.10$， $e^{\beta_2} = 1.1$， 表明家庭依恋每增加 1个单位，个体恋爱的发生比变为之前的 1.1倍。  
- 然而，$\beta_1$ 和 $\beta_2$ 的94%HDI包括0，说明**外界平均气温**和**家庭依恋情况**不能有效预测恋爱发生的概率。 

## 绘制后验回归模型(使用az.plot_hdi)  

In [12]:
log_model1_trace

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data

In [13]:
# 分别获取attachhome=3,4,5时，外界平均气温对应的索引值
attach1_index = xr.DataArray(df.query("attachhome == 3")["avgtemp"])["index"].values
attach2_index = xr.DataArray(df.query("attachhome == 4")["avgtemp"])["index"].values
attach3_index = xr.DataArray(df.query("attachhome == 5")["avgtemp"])["index"].values

# 根据上一步获得的索引值，将参数的后验预测分布分成三部分
attach1_para = log_model1_trace.sel(obs_id=attach1_index)
attach2_para = log_model1_trace.sel(obs_id=attach2_index)
attach3_para = log_model1_trace.sel(obs_id=attach3_index)

# 根据上一步获得的索引值，将参数的后验预测分布分成三部分
attach1_df = df.loc[df["attachhome"]==3]
attach2_df = df.loc[df["attachhome"]==4]
attach3_df = df.loc[df["attachhome"]==5]

In [14]:
attach1_para

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data

In [15]:
#绘制attachhome=3情况下的后验预测线性模型
#画出每个自变量对应的恋爱概率94%hdi值
az.plot_hdi(
    attach1_df.avgtemp,
    attach1_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color='#a4c2f4'
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean1 = attach1_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = attach1_df.avgtemp, 
             y= post_mean1, 
             label="attachhome=3:posterior mean", 
             color='#498cfd')
#绘制真实数据散点图
sns.scatterplot(x = attach1_df.avgtemp, 
                y= attach1_df.romantic,
                label="attachhome=3:observed data", 
                color='#498cfd', 
                alpha=0.5)


#绘制attachhome=4情况下的后验预测线性模型
#画出每个自变量对应的恋爱概率94%hdi值
az.plot_hdi(
    attach2_df.avgtemp,
    attach2_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="#fff2cc"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean2 = attach2_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = attach2_df.avgtemp, 
             y= post_mean2, 
             label="attachhome=4:posterior mean", 
             color="#fdd152")
sns.scatterplot(x = attach2_df.avgtemp, 
                y= attach2_df.romantic,
                label="attachhome=4:observed data", 
                color='#fdd152', 
                alpha=0.5)


#绘制attachhome=5情况下的后验预测线性模型
#画出每个自变量对应的恋爱概率94%hdi值
az.plot_hdi(
    attach3_df.avgtemp,
    attach3_para.posterior.pi,
    hdi_prob=0.95,
    fill_kwargs={"alpha": 0.25, "linewidth": 0},
    color="#f4cccc"
)
#得到每个自变量对应的恋爱概率均值，并使用sns.lineplot连成一条光滑的曲线
post_mean3 = attach3_para.posterior.pi.mean(("chain", "draw"))
sns.lineplot(x = attach3_df.avgtemp, 
             y= post_mean3, 
             label="attachhome=5:posterior mean", 
             color="#ec3636")
sns.scatterplot(x = attach3_df.avgtemp, 
                y= attach3_df.romantic,
                label="attachhome=5:observed data", 
                color='#ec3636', 
                alpha=0.5)


#设置图例位置
plt.legend(loc="upper right",
           bbox_to_anchor=(1.5, 1),
           fontsize=12)
sns.despine()

<Figure size 640x480 with 1 Axes>

# 对新数据进行预测&评估  
按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释

In [16]:
# 选取Serbia站点的数据
df1 = df_raw[df_raw["Site"] == "Serbia"]

# 选取本节课涉及的变量
df1 = df1[["romantic","avgtemp", "attachhome"]]

#重新编码，编码后的数据：1 = "yes"; 2 = "no"
df1["romantic"] =  np.where(df1['romantic'] == 2, 0, 1)

#设置索引
df1["index"] = range(len(df1))
df1 = df1.set_index("index")

In [17]:
# 检查数据中是否含有任何缺失值：
df1.isnull().values.any()

False

In [18]:
with log_model1:
    # 传入数据
    pm.set_data({"avgtemp": df1["avgtemp"],
                 "attachhome":df1["attachhome"],
                 "y": df1["romantic"]},
                 coords={"obs_id":range(0,len(df1))}
                 )   
    # 生成对因变量的预测
    prediction = pm.sample_posterior_predictive(log_model1_trace, 
                                                var_names=["y_est"],
                                                predictions=True,
                                                extend_inferencedata=True,
                                                random_seed=84735)

Sampling: [y_est]


In [19]:
prediction

Inference data with groups:
	> posterior
	> predictions
	> sample_stats
	> observed_data
	> constant_data
	> predictions_constant_data

## 评估结果  

**根据50-50 cut-off**  
		* 如果超过50%的结果显示 $Y=1$，我们就将 $Y$ 归类为1(恋爱)  
    * 否则将 $Y$ 归类为0  

* $Y$为每个自变量对应的二分因变量，$\hat{Y}$为对应的后验分类结果  
* 我们可以使用**混淆矩阵**(confusion matrix)来对真实结果与分类结果进行比较和评估  

    * a: 真阴性（True Negative，TN）表示被正确预测为负例的样本数  
    * b: 假阳性（False Positive，FP）表示被错误预测为正例的样本数  
    * c: 假阴性（False Negative，FN）表示被错误预测为负例的样本数  
    * d: 真阳性（True Positive，TP）表示被正确预测为正例的样本数  

|  | $\hat{Y} = 0$    | $\hat{Y} = 1$ |  
| --------  | -------- | -------- |  
| $Y=0$ | a | b |  
| $Y=1$ | c | d |

In [20]:
#stack(sample = ("chain", "draw")：将每一个X对应的4*5000个后验预测值合并到一个维度sample
#对于每一个X，需要计算其20000个值的平均值，因此将dim设置为sample
pred_pi = prediction.predictions.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
# 转换为数据框
pred_pi = pred_pi.to_dataframe()

In [21]:
#将原数据中的X 和Y存入数据框
pred_pi["romantic"] = df1["romantic"].values

#根据分类标准（50-50）生成最终的分类结果
pred_pi["romantic_pred"] = np.where(pred_pi["y_est"] >= 0.5, 1, 0)
pred_pi

,y_est,romantic,romantic_pred
obs_id,,,
0,0.49025,1,0
1,0.45855,1,0
2,0.46640,0,0
3,0.47440,0,0
4,0.49515,1,0
...,...,...,...
159,0.47745,1,0
160,0.46270,0,0
161,0.49140,1,0


In [22]:
# 使用`pd.crosstab`生成混淆矩阵，前两个值表明你需要统计的列名
# 由于要生成一个2*2的联表，需要指定行的名称和列的名称
confusion_matrix = pd.crosstab(pred_pi["romantic"], pred_pi["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

Predicted,0,1
Actual,,
0,84,2
1,77,1


In [23]:
# 计算a b c d的数量
true_positive = confusion_matrix.at[1, 1]
false_positive = confusion_matrix.at[0, 1]
true_negative = confusion_matrix.at[0, 0]
false_negative = confusion_matrix.at[1, 0]
# 代入公式
accuracy = (true_positive + false_negative) /(true_positive + false_positive + true_negative + false_negative)
sensitivity = (true_positive) /(true_positive + false_negative)
specificity = (true_negative) / (true_negative + false_positive)

print("本模型的准确性为{:.2f}，即模型正确预测的比例为{:.2%}，反映整体性能较差；".format(accuracy,accuracy))
print("本模型的敏感性为{:.2f}，指在所有实际为正例的样本中，被正确预测为正例的比例为{:.2%}，模型对于正例的预测能力较差；" .format(sensitivity,sensitivity))
print("本模型的特异性为{:.2f}，指在所有实际为负例的样本中，被正确预测为负例的比例为{:.2%}，模型对于负例的预测能力较好。".format(specificity,specificity))

本模型的准确性为0.48，即模型正确预测的比例为47.56%，反映整体性能较差；
本模型的敏感性为0.01，指在所有实际为正例的样本中，被正确预测为正例的比例为1.28%，模型对于正例的预测能力较差；
本模型的特异性为0.98，指在所有实际为负例的样本中，被正确预测为负例的比例为97.67%，模型对于负例的预测能力较好。


# 对本数据集的预测结果进行评估  
按照50-50的分类标准，计算预测结果对应的准确性、敏感性、特异性，并附上简单的文字解释

In [24]:
coords = {"obs_id": df.index}

with pm.Model(coords=coords) as log_model1:
    avgtemp = pm.MutableData("avgtemp", df.avgtemp, dims="obs_id")
    attachhome = pm.MutableData("attachhome", df.attachhome, dims="obs_id")

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * avgtemp + beta_2 * attachhome, dims="obs_id")
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=df.romantic,dims="obs_id")

    log_model1_trace = pm.sample(draws=5000,                 
                                tune=1000,                  
                                chains=4,                     
                                discard_tuned_samples= True, 
                                random_seed=84735)
    log_model1_ppc = pm.sample_posterior_predictive(log_model1_trace)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 77 seconds.
Sampling: [y_est]


In [25]:
az.plot_ppc(log_model1_ppc, num_pp_samples=50)

<Axes: xlabel='y_est / y_est'>

<Figure size 640x480 with 1 Axes>

In [26]:
log_model1_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw"))

<xarray.DataArray 'y_est' (obs_id: 108, sample: 20000)>
array([[0, 1, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 1, 0]])
Coordinates:
  * obs_id   (obs_id) int64 0 1 2 3 4 5 6 7 ... 100 101 102 103 104 105 106 107
  * sample   (sample) object MultiIndex
  * chain    (sample) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
  * draw     (sample) int64 0 1 2 3 4 5 6 ... 4993 4994 4995 4996 4997 4998 4999

In [27]:
#stack(sample = ("chain", "draw")：将每一个X对应的4*5000个后验预测值合并到一个维度sample
#对于每一个X，需要计算其20000个值的平均值，因此将dim设置为sample
pred_pi1 = log_model1_ppc.posterior_predictive.y_est.stack(sample = ("chain", "draw")).mean(dim="sample")
# 转换为数据框
pred_pi1 = pred_pi1.to_dataframe()

In [28]:
log_model1_ppc

Inference data with groups:
	> posterior_predictive
	> observed_data
	> constant_data

In [29]:
#将原数据中的X 和Y存入数据框
pred_pi1["avgtemp"] = log_model1_ppc.constant_data.avgtemp.values
pred_pi1["attachhome"] = log_model1_ppc.constant_data.attachhome.values
pred_pi1["romantic"] = log_model1_ppc.observed_data.y_est.values

#根据分类标准（50-50）生成最终的分类结果
pred_pi1["romantic_pred"] = np.where(pred_pi1["y_est"] >= 0.5, 1, 0)
pred_pi1

,y_est,avgtemp,attachhome,romantic,romantic_pred
obs_id,,,,,
0,0.49510,36.111111,4.666667,0,0
1,0.48745,36.222222,4.444444,1,0
2,0.48225,36.500000,4.444444,0,0
3,0.46590,36.527778,3.666667,0,0
4,0.48445,36.638889,4.555556,1,0
...,...,...,...,...,...
103,0.48810,36.494444,4.666667,1,0
104,0.47755,36.611111,4.333333,1,0
105,0.47145,36.805556,3.555556,1,0


In [30]:
# 使用`pd.crosstab`生成混淆矩阵，前两个值表明你需要统计的列名
# 由于要生成一个2*2的联表，需要指定行的名称和列的名称
confusion_matrix1 = pd.crosstab(pred_pi1["romantic"], pred_pi1["romantic_pred"], 
                              rownames=['Actual'], colnames=['Predicted'])
confusion_matrix1

Predicted,0
Actual,
0,57
1,51


In [31]:
# 计算a b c d的数量
if sum(pred_pi1["romantic_pred"])!=0:
    true_positive1 = confusion_matrix1.at[1, 1]
    false_positive1 = confusion_matrix1.at[0, 1]
    true_negative1 = confusion_matrix1.at[0, 0]
    false_negative1 = confusion_matrix1.at[1, 0]
else:
    true_positive1 = 0
    false_positive1 = 0
    true_negative1 = confusion_matrix1.at[0, 0]
    false_negative1 = confusion_matrix1.at[1, 0]   
# 代入公式
accuracy1 = (true_positive1 + false_negative1) /(true_positive1 + false_positive1 + true_negative1 + false_negative1)
sensitivity1 = (true_positive1) /(true_positive1 + false_negative1)
specificity1 = (true_negative1) / (true_negative1 + false_positive1)

print("本模型的准确性为{:.2f}，即模型正确预测的比例为{:.2%}，反映整体性能较差；".format(accuracy1,accuracy1))
print("本模型的敏感性为{:.2f}，指在所有实际为正例的样本中，被正确预测为正例的比例为{:.2%}，模型对于正例的预测能力较差；" .format(sensitivity1,sensitivity1))
print("本模型的特异性为{:.2f}，指在所有实际为负例的样本中，被正确预测为负例的比例为{:.2%}，模型对于负例的预测能力较好。".format(specificity1,specificity1))

本模型的准确性为0.47，即模型正确预测的比例为47.22%，反映整体性能较差；
本模型的敏感性为0.00，指在所有实际为正例的样本中，被正确预测为正例的比例为0.00%，模型对于正例的预测能力较差；
本模型的特异性为1.00，指在所有实际为负例的样本中，被正确预测为负例的比例为100.00%，模型对于负例的预测能力较好。


# 模型比较  
与课上的模型(log_model2:自变量为回避倾向; log_model3:自变量为性别)进行模型比较  

## 导入课上的模型  
log_model2:自变量为回避倾向

In [32]:
with pm.Model() as log_model2:
    # 此处对coords的定义方式进行了更改，因为后续我们需要进行对新数据的预测
    # 因此将维度定义成可更改的
    log_model2.add_coord('obs_id',df.index, mutable=True)
    avoidance = pm.MutableData("avoidance", df.avoidance_r, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=0.5)           #定义beta_1
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_1 * avoidance, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [33]:
#===========================
#     注意！！！以下代码可能需要运行1-2分钟左右
#===========================
with log_model2:
    # MCMC 近似后验分布
    log_model2_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 20 seconds.


log_model3:自变量为性别

In [34]:
##---------------------------------------------------------------------------
#      提示：参照之前的代码与先验定义，对...中的内容进行修改
#---------------------------------------------------------------------------
coords = {"obs_id": df.index}

with pm.Model() as log_model3:
    log_model3.add_coord('obs_id',df.index, mutable=True)
    sex= pm.MutableData("sex", df.sex, dims="obs_id")
    y = pm.MutableData('y', df.romantic, dims = 'obs_id')

    #先验
    beta_0 = pm.Normal("beta_0", mu=0, sigma=0.5)          #定义beta_0          
    beta_2 = pm.Normal("beta_2", mu=0, sigma=0.5)           #定义beta_2
    #线性关系
    mu = pm.Deterministic("mu", beta_0 + beta_2 *sex, dims="obs_id")
    #注意此处使用了Logistic sigmoid function：pm.math.invlogit
    #相当于进行了如下计算 (1 / (1 + exp(-mu))
    pi = pm.Deterministic("pi", pm.math.invlogit(mu), dims="obs_id")
    #似然
    likelihood = pm.Bernoulli("y_est",p=pi, observed=y,dims="obs_id")

In [35]:
#===========================
#     注意！！！以下代码可能需要运行1-2分钟左右
#===========================
with log_model3:
    # MCMC 近似后验分布
    log_model3_trace = pm.sample(
                                draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                                tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                                chains=4,                     # 链数
                                discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                                random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_2]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 23 seconds.


## 比较

In [36]:
# 计算模型对数似然值以便进一步计算 ELPD. 
with log_model1:
    pm.compute_log_likelihood(log_model1_trace)

with log_model2:
   pm.compute_log_likelihood(log_model2_trace)

with log_model3:
   pm.compute_log_likelihood(log_model3_trace)

In [37]:
comparison_list = {
    "model1(mine)":log_model1_trace,
    "model2(t's avoidance)":log_model2_trace,
    "model3(t's sex)":log_model3_trace
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model3(t's sex),0,-76.050362,1.419829,0.000000,1.000000e+00,0.620354,0.000000,False,log
model2(t's avoidance),1,-76.303490,1.806491,0.253127,5.551115e-17,0.767736,0.613830,False,log
model1(mine),2,-76.384289,1.777293,0.333926,4.440892e-16,0.658629,0.445265,False,log
